In [5]:
# Importing the libraries
import pandas as pd
import kagglehub
import os
import numpy as np

In [2]:
path = kagglehub.dataset_download("chavindudulaj/vehicle-maintenance-data")
print(path)
# Load the dataset
csv_path = os.path.join(path, "vehicle_maintenance_data.csv")
data = pd.read_csv(csv_path)
print(data.head())

Using Colab cache for faster access to the 'vehicle-maintenance-data' dataset.
/kaggle/input/vehicle-maintenance-data
  Vehicle_Model  Mileage Maintenance_History  Reported_Issues  Vehicle_Age  \
0         Truck    58765                Good                0            4   
1           Van    60353             Average                1            7   
2           Bus    68072                Poor                0            2   
3           Bus    60849             Average                4            5   
4           Bus    45742                Poor                5            1   

  Fuel_Type Transmission_Type  Engine_Size  Odometer_Reading  \
0  Electric         Automatic         2000             28524   
1  Electric         Automatic         2500            133630   
2  Electric         Automatic         1500             34022   
3    Petrol         Automatic         2500             81636   
4    Petrol            Manual         2000             97162   

  Last_Service_Date Warranty

In [3]:
imbalance = data['Need_Maintenance'].value_counts(normalize=True)
imbalance

,proportion
Need_Maintenance,
1,0.80996
0,0.19004


In [6]:
corr_matrix = data.select_dtypes(include=[np.number]).corr()
corr_matrix

,Mileage,Reported_Issues,Vehicle_Age,Engine_Size,Odometer_Reading,Insurance_Premium,Service_History,Accident_History,Fuel_Efficiency,Need_Maintenance
Mileage,1.000000,0.004311,0.000341,0.006994,-0.002929,-0.004834,-0.001166,0.002185,0.012465,0.000704
Reported_Issues,0.004311,1.000000,-0.004607,-0.000349,0.000119,0.000248,-0.002567,0.007023,0.003954,0.389273
Vehicle_Age,0.000341,-0.004607,1.000000,-0.006184,-0.000556,0.003263,0.001983,-0.005237,-0.004572,0.000806
Engine_Size,0.006994,-0.000349,-0.006184,1.000000,-0.000369,-0.009805,0.009459,0.000589,-0.000645,-0.000326
Odometer_Reading,-0.002929,0.000119,-0.000556,-0.000369,1.000000,-0.009439,0.000107,0.001599,0.004281,0.007876
Insurance_Premium,-0.004834,0.000248,0.003263,-0.009805,-0.009439,1.000000,-0.007262,0.000120,-0.003988,0.004775
Service_History,-0.001166,-0.002567,0.001983,0.009459,0.000107,-0.007262,1.000000,0.003199,0.004532,0.104155
Accident_History,0.002185,0.007023,-0.005237,0.000589,0.001599,0.000120,0.003199,1.000000,0.002825,0.080590
Fuel_Efficiency,0.012465,0.003954,-0.004572,-0.000645,0.004281,-0.003988,0.004532,0.002825,1.000000,0.001874
Need_Maintenance,0.000704,0.389273,0.000806,-0.000326,0.007876,0.004775,0.104155,0.080590,0.001874,1.000000
